In [ ]:
import os
import shutil
import random
from tqdm import tqdm

In [ ]:
def create_subset(source, dest, num_train_images, num_val_images):
    os.makedirs(dest)

    shutil.copy2(os.path.join(source, "categories_places365.txt"), dest)

    a = num_train_images // 365
    b = num_train_images % 365
    train_images_per_class = [a] * 365
    for i in range(b):
        train_images_per_class[i] += 1

    source_train_folder = os.path.join(os.path.join(source, "data_large_standard"))
    source_val_folder = os.path.join(os.path.join(source, "val_large"))
    dest_train_folder = os.path.join(os.path.join(dest, "data_large_standard"))
    dest_val_folder = os.path.join(os.path.join(dest, "val_large"))


    tqdm_letter_folders = tqdm(os.listdir(source_train_folder), unit="letter")
    tqdm_letter_folders.set_description(f"Copy train images")
    index = 0
    source_train_file = open(os.path.join(source, "places365_train_standard.txt"))
    dest_train_file = open(os.path.join(dest, "places365_train_standard.txt"), "w+")
    rows_train_dict = {}
    for row in source_train_file.readlines():
        key = (row.split()[0])[1:]
        rows_train_dict[key] = row
    for letter in tqdm_letter_folders:
        index = create_subset_helper(source_train_folder, dest_train_folder, [letter], train_images_per_class, index, rows_train_dict, dest_train_file)
    source_train_file.close()
    dest_train_file.close()
    
    # Copy val images
    source_val_file = open(os.path.join(source, "places365_val.txt"))
    dest_val_file = open(os.path.join(dest, "places365_val.txt"), "w+")

    rows_val = [row for row in source_val_file.readlines()]

    val_images = os.listdir(source_val_folder)
    selected_val_images = random.sample(val_images, min(len(val_images), num_val_images))
    os.makedirs(dest_val_folder)

    for img in selected_val_images:
        src_val_img_path = os.path.join(source_val_folder, img)
        dest_val_img_path = os.path.join(dest_val_folder, img)
        shutil.copy2(src_val_img_path, dest_val_img_path)

        row = next(row for row in rows_val if img in row)
        dest_val_file.write(row)
    
    source_val_file.close()
    dest_val_file.close()

def create_subset_helper(source, dest, path_parts, train_images_per_class, index, rows_train_dict, dest_train_file):
    current_folder = os.path.join(source, *path_parts)
    current_folder_content = os.listdir(current_folder)

    if os.path.isfile(os.path.join(current_folder, current_folder_content[0])):
        # Create corresponding class folder in destination
        source_class_path = current_folder
        dest_class_path = os.path.join(os.path.join(dest, *path_parts))
        os.makedirs(dest_class_path)
        
        # List all images in the class folder
        images = os.listdir(source_class_path)
        
        # Randomly sample images (or take fewer if not enough images)
        selected_images = random.sample(images, min(len(images), train_images_per_class[index]))
        
        # Copy selected images to destination folder
        for img in selected_images:
            src_img_path = os.path.join(source_class_path, img)
            dest_img_path = os.path.join(dest_class_path, img)
            shutil.copy2(src_img_path, dest_img_path)

            dest_train_file.write(rows_train_dict["/".join(path_parts + [img])])
        
        return index + 1
    
    for folder in current_folder_content:
        index = create_subset_helper(source, dest, path_parts + [folder], train_images_per_class, index, rows_train_dict, dest_train_file)
    
    return index

In [ ]:
create_subset(r"C:\Users\mariu\Documents\Development\Datasets\Places365", r"C:\Users\mariu\Documents\Studium\Praktikum\Places365_Evaluation_Subset", 65536, 4096)